In [0]:
%pip install hdbscan

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 38.7 MB/s eta 0:00:00
  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Not uninstalling numpy at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-a69f6608-6af7-46de-84a1-f6a78da6d459
    Can't uninstall 'numpy'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
databricks-feature-store 0.14.3 requires pyspark<4,>=3.1.2, which is not installed.
ydata-profiling 4.2.0 requires numpy<1.24,>

In [0]:
dbutils.library.restartPython()

In [0]:
# import cuml
# from cuml import DBSCAN

In [0]:
import numpy as np
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.get_bucket('gcs-dsci-fryou-fy-dev-prd')

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [0]:
path = "gs://gcs-dsci-fryou-fy-dev-prd/catalog_vqvae/vqvae_catalog_embeddings_2025-03-01"
df = spark.read.parquet(path)

In [0]:
taxonomy = spark.sql("""select distinct catalog_id, sscat_id, scat_id, sscat from gold.product_info""")


In [0]:
intersected_df = df.join(
    taxonomy, 
    on="catalog_id", 
    how="inner"
).select("catalog_id", "sscat_id", "sscat", "search_catalog_embedding")

In [0]:
df_sarees = intersected_df.filter(intersected_df.sscat == "Jeans")
pandas_df = df_sarees.select('search_catalog_embedding').toPandas()

In [0]:
pandas_df.count()

search_catalog_embedding    71324
dtype: int64

In [0]:
catalog_embedding_1m = np.array(
    pandas_df['search_catalog_embedding'].tolist(),
    dtype=np.float32
)

# display(catalog_embedding_1m)

In [0]:
# dbscan_model = DBSCAN(eps=0.05, min_samples=1000, metric='euclidean')
# dbscan_model = DBSCAN(eps=0.05, min_samples=1000, verbose=6, algorithm='brute')


# dbscan_model.fit(catalog_embedding_1m)

In [0]:
from hdbscan import HDBSCAN

hdbscan = HDBSCAN(
    min_cluster_size=65,     
    min_samples=10,          
    metric='euclidean'
)


In [0]:
labels = hdbscan.fit_predict(catalog_embedding_1m)

In [0]:
from collections import Counter

cluster_counts = Counter(labels)
print(cluster_counts)

Counter({0: 71180, 1: 79, -1: 65})


In [0]:
import numpy as np

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

print(f"Estimated number of clusters: {n_clusters}")
print(f"Estimated number of noise points: {n_noise}")

Estimated number of clusters: 2
Estimated number of noise points: 65


In [0]:
pandas_df['cluster'] = labels
sdf_withcluster = spark.createDataFrame(pandas_df)
result = sdf_withcluster.join(df, on="search_catalog_embedding", how="inner")
result = result.drop("search_catalog_embedding")

In [0]:
result.filter(result.cluster == -1).select("catalog_id").rdd.flatMap(lambda x: x).collect()

[115910672,
 163898281,
 86180251,
 134010316,
 142840220,
 123600628,
 145098000,
 26664093,
 144905245,
 124434474,
 115910676,
 156873667,
 135189696,
 116133483,
 117194643,
 127205266,
 161012942,
 136830219,
 125913466,
 131407376,
 144987294,
 540880,
 116882232,
 144809044,
 114189060,
 135450793,
 157237550,
 161267570,
 145745735,
 165542393,
 142825092,
 83111752,
 118922914,
 113447672,
 115910652,
 156027098,
 43250148,
 144785291,
 82337258,
 33273860,
 160583160,
 144262828,
 157827056,
 151640631,
 152460389,
 540877,
 131718609,
 19529267,
 162872505,
 143274909,
 119686214,
 127211358,
 164779832,
 131752306,
 156399887,
 3804429,
 160429645,
 118026963,
 123602561,
 131811871,
 143888038,
 119479043,
 23041870,
 132476390,
 93316418]

# Kmeans

In [0]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=50, random_state=42, verbose=6)
clusters = model.fit_predict (catalog_embedding_1m)
pandas_df['cluster'] = clusters
sdf_withcluster = spark.createDataFrame(pandas_df)
result_50 = sdf_withcluster.join(df, on="search_catalog_embedding", how="inner")


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ed66ce312d0>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Initialization complete
Iteration 0, inertia 17268.3671875.
Iteration 1, inertia 11833.2744140625.
Iteration 2, inertia 11603.283203125.
Iteration 3, inertia 11508.80078125.
Iteration 4, inertia 11444.712890625.
Iteration 5, inertia 11393.0673828125.
Iteration 6, inertia 11360.4833984375.
Iteration 7, inertia 11333.2275390625.
Iteration 8, inertia 11308.3408203125.
Iteration 9, inertia 11289.3505859375.
Iteration 10, inertia 11275.6240234375.
Iteration 11, inertia 11265.810546875.
Iteration 12, inertia 11258.7197265625.
Iteration 13, inertia 11253.18359375.
Iteration 14, inertia 11248.73046875.
Iteration 15, inertia 11245.3388671875.
Iteration 16, inertia 11242.7607421875.
Iteration 17, inertia 11239.888671875.
Iteration 18, inertia 11235.42578125.
Iteration 19, inertia 11230.4462890625.
Iteration 20, inertia 11227.37890625.
Iteration 21, inertia 11225.17578125.
Iteration 22, inertia 11222.462890625.
Iteration 23, inertia 11219.1005859375.
Iteration 24, inertia 11214.8759765625.
Iterat

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ed66ce312d0>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Initialization complete
Iteration 0, inertia 16976.53515625.
Iteration 1, inertia 11772.2265625.
Iteration 2, inertia 11519.0859375.
Iteration 3, inertia 11421.0234375.
Iteration 4, inertia 11364.7509765625.
Iteration 5, inertia 11322.470703125.
Iteration 6, inertia 11289.3427734375.
Iteration 7, inertia 11260.916015625.
Iteration 8, inertia 11242.431640625.
Iteration 9, inertia 11231.3359375.
Iteration 10, inertia 11224.0625.
Iteration 11, inertia 11218.0810546875.
Iteration 12, inertia 11213.5380859375.
Iteration 13, inertia 11209.703125.
Iteration 14, inertia 11206.5048828125.
Iteration 15, inertia 11203.3193359375.
Iteration 16, inertia 11200.0810546875.
Iteration 17, inertia 11196.2197265625.
Iteration 18, inertia 11191.255859375.
Iteration 19, inertia 11186.8046875.
Iteration 20, inertia 11183.8125.
Iteration 21, inertia 11181.1083984375.
Iteration 22, inertia 11178.9814453125.
Iteration 23, inertia 11177.4326171875.
Iteration 24, inertia 11176.3466796875.
Iteration 25, inertia 1

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ed5c97a9240>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


...

Initialization complete
Iteration 0, inertia 17248.69140625.
Iteration 1, inertia 11733.0712890625.
Iteration 2, inertia 11456.962890625.
Iteration 3, inertia 11387.642578125.
Iteration 4, inertia 11353.5498046875.
Iteration 5, inertia 11327.8046875.
Iteration 6, inertia 11306.2275390625.
Iteration 7, inertia 11288.3515625.
Iteration 8, inertia 11273.76171875.
Iteration 9, inertia 11261.537109375.
Iteration 10, inertia 11251.580078125.
Iteration 11, inertia 11243.3974609375.
Iteration 12, inertia 11236.4970703125.
Iteration 13, inertia 11230.2470703125.
Iteration 14, inertia 11225.37109375.
Iteration 15, inertia 11221.615234375.
Iteration 16, inertia 11217.7275390625.
Iteration 17, inertia 11212.6318359375.
Iteration 18, inertia 11206.54296875.
Iteration 19, inertia 11202.1357421875.
Iteration 20, inertia 11199.609375.
Iteration 21, inertia 11197.107421875.
Iteration 22, inertia 11194.7685546875.
Iteration 23, inertia 11192.9736328125.
Iteration 24, inertia 11191.62109375.
Iteration 25

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ed5c97a9240>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Initialization complete
Iteration 0, inertia 17550.9296875.
Iteration 1, inertia 11762.6943359375.
Iteration 2, inertia 11502.5751953125.
Iteration 3, inertia 11388.3388671875.
Iteration 4, inertia 11324.8349609375.
Iteration 5, inertia 11281.6826171875.
Iteration 6, inertia 11246.8447265625.
Iteration 7, inertia 11216.109375.
Iteration 8, inertia 11195.3076171875.
Iteration 9, inertia 11181.7998046875.
Iteration 10, inertia 11171.4755859375.
Iteration 11, inertia 11164.1884765625.
Iteration 12, inertia 11159.33984375.
Iteration 13, inertia 11156.1376953125.
Iteration 14, inertia 11153.5546875.
Iteration 15, inertia 11151.65625.
Iteration 16, inertia 11150.3359375.
Iteration 17, inertia 11149.34375.
Iteration 18, inertia 11148.3193359375.
Iteration 19, inertia 11147.1328125.
Iteration 20, inertia 11145.9599609375.
Iteration 21, inertia 11144.8544921875.
Iteration 22, inertia 11143.7666015625.
Iteration 23, inertia 11142.8330078125.
Iteration 24, inertia 11142.15625.
Iteration 25, inert

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ed5c97a9240>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Initialization complete
Iteration 0, inertia 17657.619140625.
Iteration 1, inertia 11830.7255859375.
Iteration 2, inertia 11504.16796875.
Iteration 3, inertia 11372.66015625.
Iteration 4, inertia 11316.921875.
Iteration 5, inertia 11285.4072265625.
Iteration 6, inertia 11262.662109375.
Iteration 7, inertia 11242.3017578125.
Iteration 8, inertia 11223.91015625.
Iteration 9, inertia 11207.796875.
Iteration 10, inertia 11195.1669921875.
Iteration 11, inertia 11186.626953125.
Iteration 12, inertia 11181.013671875.
Iteration 13, inertia 11176.697265625.
Iteration 14, inertia 11173.076171875.
Iteration 15, inertia 11170.53125.
Iteration 16, inertia 11168.90234375.
Iteration 17, inertia 11167.8310546875.
Iteration 18, inertia 11166.9287109375.
Iteration 19, inertia 11166.1494140625.
Iteration 20, inertia 11165.517578125.
Iteration 21, inertia 11165.08203125.
Iteration 22, inertia 11164.6884765625.
Iteration 23, inertia 11164.4052734375.
Iteration 24, inertia 11164.1962890625.
Iteration 25, in

In [0]:
intersected_result_50 = result_50.join(
    taxonomy, 
    on="catalog_id", 
    how="inner"
).select("cluster", "catalog_id", "sscat_id", "sscat")

In [0]:
catalog_id_list

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5509296356043018>, line 1
----> 1 catalog_id_list

NameError: name 'catalog_id_list' is not defined

In [0]:
catalog_id_list = intersected_result_50.filter(intersected_result_50.cluster == 20).select("catalog_id").rdd.flatMap(lambda x: x).collect()
display_images(catalog_id_list)

In [0]:
%run
./display_images

# Generate Data